In [0]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

In [0]:
sub_2014 = spark.read.parquet("/mnt/lsde/group05/submissions_tree_shaking/2014-2.parquet")

In [0]:
document_assembler = DocumentAssembler().setInputCol("title").setOutputCol("document").setCleanupMode("shrink")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
normalizer = Normalizer().setInputCols(["token"]).setOutputCol("normalized")
stopwords_cleaner = StopWordsCleaner().setInputCols("normalized").setOutputCol("cleanTokens").setCaseSensitive(False)
stemmer = Stemmer().setInputCols(["cleanTokens"]).setOutputCol("stem")
finisher = Finisher().setInputCols(["stem"]).setOutputCols(["tokens"]).setOutputAsArray(True).setCleanAnnotations(False)
nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher])
nlp_model = nlp_pipeline.fit(sub_2014)
processed_df = nlp_model.transform(sub_2014)

In [0]:
processed_df.select('id','tokens').show()

+------+--------------------+
 id| tokens|
+------+--------------------+
1yjc0q|[eli, situat, kie...|
1yjc5i|[liveleakcom, ukr...|
1yjc9m| [russian, fox]|
1yjcc5|[turmoil, ukrain,...|
1yjcet| [bash, putin]|
1yjcht|[unrest, ukrain, ...|
1yjcko|[fight, kiev, ukr...|
1yjcn2|[find, proof, vla...|
1yjcsl|[ukrain, agreem, ...|
1yjd0u|[agreem, settlem,...|
1yjd4x|[sever, portrait,...|
1yjd9x|[russia, prepar, ...|
1yjdhg|[ukrainian, ми, в...|
1yjdie|[d, print, badg, ...|
1yjdtb|[vladimir, putin,...|
1yjdtk|[abandon, circu, ...|
1yjdz2|[polic, defector,...|
1yje03|[racial, polit, d...|
1yje2e|[deal, sign, end,...|
1yje6e|[russia, expert, ...|
+------+--------------------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=500, minDF=3.0)
# train the model
cv_model = cv.fit(processed_df)
# transform the data. Output column name will be features.
vectorized_tokens = cv_model.transform(processed_df)

In [0]:
from pyspark.ml.clustering import LDA
num_topics = 2
lda = LDA(k=num_topics, maxIter=1)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -436089.83645132894
The upper bound on perplexity: 5.754606516298735

In [0]:
# extract vocabulary from CountVectorizer
vocab = cv_model.vocabulary
topics = model.describeTopics()   
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
russian
russia
ukrain
putin
olymp
vladimir
gai
sochi
invas
attack
*************************
topic: 1
*************************
ukrain
russia
russian
protest
u
putin
new
presid
kiev
olymp
*************************